In [1]:
%matplotlib inline


# Markov chains

Here we look at how to formulate expected threat in terms of a Markov chain.

In [2]:
import numpy as np

## Setting up the matrix
We first set up the pass matrix *A* and the goal vector $\vec{g}$ to solve:
$$ \vec{xT} = A\vec{xT} + \vec{g}$$

whith $\vec{xT} = (xT_M, xT_B, xT_W)^t$ where $xT_i$ is the probability of ending scoring a goal (expected threat) starting at position $i\in \{M, B, W\}$ where M: MidField, B: Box, W: Wing.


In [3]:
#Pass matrix
A = np.matrix([[0.25, 0.20, 0.1], [0.1, 0.25, 0.2], [0.1, 0.1, 0.25]])
#Goal vector
g = np.transpose(np.matrix([0.05, 0.15, 0.05]))

## Linear algebra method

Here we solve $(I-A)\vec{xT} = \vec{g}$



In [4]:
xT1 = np.linalg.solve(np.identity(3) - A, g)

print('Expected Threat')
print('MidField, Box, Wing')
print(np.transpose(xT1))

Expected Threat
MidField, Box, Wing
[[0.14991763 0.25205931 0.12026359]]


## Iterative method

Here we iterate $\vec{xT}_{t+1} = A \vec{xT}_{t} + \vec{g}$
to update through each move of the ball



In [6]:
xT2 = np.zeros((3,1))
iterations = 10

for t in range(iterations):
   #print(np.matmul(A,xT2) + g)
   xT2 = np.matmul(A, xT2) + g

print('Expected Threat')
print('MidField, Box, Wing')
print(np.transpose(xT2))

Expected Threat
MidField, Box, Wing
[[0.14966911 0.25182476 0.12007973]]


## Simulation method

Here we simulate *num_sim* possessions, starting from each of the three areas.



In [7]:
num_sims = 10
xT3 = np.zeros(3) 

description = {0: 'MidField', 1: 'Wing', 2: 'Box' }

for i in range(len(description)):
    num_goals = 0

    print('---------------')
    print('Start from ' + description[i] )
    print('---------------')

    for n in range(num_sims):
        
        ballinplay=True
        #Initial state is i
        s = i
        describe_possession=''
        
        while ballinplay:
            r=np.random.rand()
            
            # Make commentary text
            describe_possession = describe_possession + ' - ' + description[s]
            
            
            #Cumulative sum of in play probabilities
            c_sum=np.cumsum(A[s,:])
            new_s = np.sum(r>c_sum)  
            if new_s>2:
                #Ball is either goal or out of play
                ballinplay=False
                if r < g[s] + c_sum[0,2]:
                    #Its a goal!
                    num_goals = num_goals + 1
                    describe_possession = describe_possession + ' - Goal!'
                else:
                    describe_possession = describe_possession + ' - Out of play'
            s = new_s  
            
        print(describe_possession)  
            
    xT3[i] = num_goals/num_sims


print('\n\n---------------')
print('Expected Threat')
print('MidField, Box, Wing')
print(xT3)

---------------
Start from MidField
---------------
 - MidField - Wing - Goal!
 - MidField - Out of play
 - MidField - Out of play
 - MidField - MidField - MidField - Out of play
 - MidField - MidField - Box - Wing - Box - Goal!
 - MidField - Box - Out of play
 - MidField - MidField - Out of play
 - MidField - Out of play
 - MidField - Wing - Out of play
 - MidField - Out of play
---------------
Start from Wing
---------------
 - Wing - Goal!
 - Wing - Out of play
 - Wing - Out of play
 - Wing - Box - Out of play
 - Wing - Wing - Out of play
 - Wing - Out of play
 - Wing - Wing - Out of play
 - Wing - Wing - Box - Out of play
 - Wing - MidField - Out of play
 - Wing - Out of play
---------------
Start from Box
---------------
 - Box - Goal!
 - Box - Out of play
 - Box - Box - Out of play
 - Box - Out of play
 - Box - Out of play
 - Box - Box - Out of play
 - Box - Box - Out of play
 - Box - MidField - Out of play
 - Box - Out of play
 - Box - Out of play


---------------
Expected Thre

If we run it for many more simulations we achieve same results:

In [8]:
num_sims = 10000
xT3 = np.zeros(3) 

description = {0: 'MidField', 1: 'Wing', 2: 'Box' }

for i in range(len(description)):
    num_goals = 0

    print('---------------')
    print('Start from ' + description[i] )
    print('---------------')

    for n in range(num_sims):
        
        ballinplay=True
        #Initial state is i
        s = i
        describe_possession=''
        
        while ballinplay:
            r=np.random.rand()
            
            # Make commentary text
            describe_possession = describe_possession + ' - ' + description[s]
            
            
            #Cumulative sum of in play probabilities
            c_sum=np.cumsum(A[s,:])
            new_s = np.sum(r>c_sum)  
            if new_s>2:
                #Ball is either goal or out of play
                ballinplay=False
                if r < g[s] + c_sum[0,2]:
                    #Its a goal!
                    num_goals = num_goals + 1
                    describe_possession = describe_possession + ' - Goal!'
                else:
                    describe_possession = describe_possession + ' - Out of play'
            s = new_s  
            
#         print(describe_possession)  
            
    xT3[i] = num_goals/num_sims


print('\n\n---------------')
print('Expected Threat')
print('MidField, Box, Wing')
print(xT3)

---------------
Start from MidField
---------------
---------------
Start from Wing
---------------
---------------
Start from Box
---------------


---------------
Expected Threat
MidField, Box, Wing
[0.1529 0.2607 0.119 ]
